In [61]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
import pandas as pd

import time

song = []
rank = []
year = []
artist = []


for i in range(1975,2020):
    print(i)
    
#     page = Request("https://www.billboard.com/charts/year-end/2019/hot-100-songs",headers={'User-Agent': 'Mozilla/5.0'})
    page = Request("https://www.billboard.com/charts/year-end/" + str(i) + "/hot-100-songs",headers={'User-Agent': 'Mozilla/5.0'})

    html = urlopen(page).read()
    page_soup = soup(html,"html.parser")

    #splits the page into the 100 artists
    containers = page_soup.main.findAll("div",{"class":"ye-chart-item__primary-row"})
    
    #Sleep because if not denies too many requests
    time.sleep(2)


    #iterates through the 100 artists
    for container in containers:

        rank.append(int(container.contents[1].text.strip()))
        song.append(container.find("div",{"class":"ye-chart-item__title"}).text.strip())
        artist.append(container.find("div",{"class":"ye-chart-item__artist"}).text.strip())
        year.append(int(i))
        
data = pd.DataFrame()

data['song'] = song
data['artist'] = artist
data['rank'] = rank
data['year'] = year


# data.astype({'year': 'int32'}).dtypes
# data.astype({'rank': 'int32'}).dtypes
data.info()
    
    





1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4285 entries, 0 to 4284
Data columns (total 4 columns):
song      4285 non-null object
artist    4285 non-null object
rank      4285 non-null int64
year      4285 non-null int64
dtypes: int64(2), object(2)
memory usage: 134.0+ KB


In [100]:

import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials


SPOTIPY_CLIENT_ID='dfc90bae743943689c070c2f0b7fd7e0'
SPOTIPY_CLIENT_SECRET='477f42668a624f6e875a5399643b8c00'

# passes in the credentials for the cleint credential flow
client_credentials_manager = SpotifyClientCredentials(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

results = sp.search(q='old town road', type='track',limit = 1)



    
results['tracks']['items'][0]
# for i,row in data.iterrows():
    

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jVv8c5Fj3E9VhNjxT4snq'},
    'href': 'https://api.spotify.com/v1/artists/7jVv8c5Fj3E9VhNjxT4snq',
    'id': '7jVv8c5Fj3E9VhNjxT4snq',
    'name': 'Lil Nas X',
    'type': 'artist',
    'uri': 'spotify:artist:7jVv8c5Fj3E9VhNjxT4snq'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
 

In [109]:
cur = data.loc[data['year'] == 2019]
for i,row in cur.iterrows():
    result = sp.search(q=row['song'] +' ' + row['artist'], type='track',limit = 1)
    print(result['tracks']['items'][0])
    for key in result['tracks']['items'][0]:
        print(key)
    break
    

IndexError: list index out of range